In [33]:
import pandas as pd
import numpy as np 
import altair as alt

alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [34]:
def retrieve_appregate_data():
    county_infection_data = pd.read_csv('relevant_data/us-counties.csv')
    county_population_data = pd.read_csv('relevant_data/county-population.csv')
    
    county_infection_popu_data = county_infection_data.merge(
        county_population_data, left_on=['county', 'state'], right_on=['county', 'state']
    )
    
    return county_infection_popu_data

In [35]:
data = retrieve_appregate_data()

In [36]:
data

,date,county,state,fips,cases,deaths,population,land_area_km,density_km
0,2020-01-21,Snohomish,Washington,53061.0,1,0,822083,5413,151.872
1,2020-01-22,Snohomish,Washington,53061.0,1,0,822083,5413,151.872
2,2020-01-23,Snohomish,Washington,53061.0,1,0,822083,5413,151.872
3,2020-01-24,Snohomish,Washington,53061.0,1,0,822083,5413,151.872
4,2020-01-25,Snohomish,Washington,53061.0,1,0,822083,5413,151.872
...,...,...,...,...,...,...,...,...,...
100503,2020-04-30,Washington,Kansas,20201.0,1,0,5758,2326,2.475
100504,2020-04-30,Fergus,Montana,30027.0,1,0,11442,11238,1.018
100505,2020-04-30,Hitchcock,Nebraska,31087.0,1,0,2872,1839,1.562
100506,2020-04-30,Garza,Texas,48169.0,1,0,6528,2321,2.813


In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100508 entries, 0 to 100507
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   date          100508 non-null  object 
 1   county        100508 non-null  object 
 2   state         100508 non-null  object 
 3   fips          100508 non-null  float64
 4   cases         100508 non-null  int64  
 5   deaths        100508 non-null  int64  
 6   population    100508 non-null  int64  
 7   land_area_km  100508 non-null  int64  
 8   density_km    100508 non-null  float64
dtypes: float64(2), int64(4), object(3)
memory usage: 7.7+ MB


In [38]:
def count_days(series):
    time_series = pd.to_datetime(series)
    first_date = time_series.iloc[0]
    last_date = time_series.iloc[-1]
    
    return (last_date - first_date).days

def case_count_county_days(series, days = 20):
    if len(series) < days:
        return series.iloc[-1]
    else:
        return series.iloc[days - 1]

In [39]:
def group_data(data):
    grouped_data = data.groupby(['state', 'county']).agg(
        population=('population', lambda x: x.iloc[0]),
        density_km=('density_km', lambda x: x.iloc[0]),
        days_counted=('date', count_days),
        case_sum=('cases', lambda x: x.iloc[-1]),
        death_sum=('deaths', lambda x: x.iloc[-1]),
        case_count_20_days=('cases', case_count_county_days),
        death_count_20_days=('deaths', case_count_county_days)
    )
    
    grouped_data = grouped_data[grouped_data['days_counted'] >= 20]
    grouped_data['confirmed_infection(%_of_pop)'] = grouped_data['case_sum']/grouped_data['population']*100
    grouped_data['death_perc(%_of_case)'] = grouped_data['death_sum']/grouped_data['case_sum']*100
    grouped_data['avg_daily_case_increase(%_of_pop)'] = grouped_data['confirmed_infection(%_of_pop)']/grouped_data['days_counted']
    grouped_data = grouped_data[grouped_data['confirmed_infection(%_of_pop)'] != float("inf")]
    grouped_data['case_count_20_days(%_of_pop)'] = grouped_data['case_count_20_days']/grouped_data['population']*100
    grouped_data['death_count_20_days(%_of_case)'] = grouped_data['death_count_20_days']/grouped_data['case_count_20_days']*100
    
    return grouped_data.reset_index()

In [40]:
grouped_data = group_data(data)

In [41]:
grouped_data

,state,county,population,density_km,days_counted,case_sum,death_sum,case_count_20_days,death_count_20_days,confirmed_infection(%_of_pop),death_perc(%_of_case),avg_daily_case_increase(%_of_pop),case_count_20_days(%_of_pop),death_count_20_days(%_of_case)
0,Alabama,Autauga,54571,35.436,37,42,4,19,1,0.076964,9.523810,0.002080,0.034817,5.263158
1,Alabama,Baldwin,182265,44.261,47,174,3,25,1,0.095465,1.724138,0.002031,0.013716,4.000000
2,Alabama,Barbour,27457,11.979,27,39,1,29,0,0.142040,2.564103,0.005261,0.105620,0.000000
3,Alabama,Bibb,22915,14.215,31,42,0,26,0,0.183286,0.000000,0.005912,0.113463,0.000000
4,Alabama,Blount,57322,34.325,36,37,0,15,0,0.064548,0.000000,0.001793,0.026168,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2546,Wyoming,Sublette,10368,0.820,33,1,0,1,0,0.009645,0.000000,0.000292,0.009645,0.000000
2547,Wyoming,Sweetwater,45267,1.676,37,11,0,7,0,0.024300,0.000000,0.000657,0.015464,0.000000
2548,Wyoming,Teton,21675,2.088,43,65,1,40,0,0.299885,1.538462,0.006974,0.184544,0.000000
2549,Wyoming,Uinta,21025,3.899,28,6,0,6,0,0.028537,0.000000,0.001019,0.028537,0.000000


In [42]:
grouped_data.sample(10)

,state,county,population,density_km,days_counted,case_sum,death_sum,case_count_20_days,death_count_20_days,confirmed_infection(%_of_pop),death_perc(%_of_case),avg_daily_case_increase(%_of_pop),case_count_20_days(%_of_pop),death_count_20_days(%_of_case)
1923,South Carolina,Charleston,396484,112.734,55,444,7,52,1,0.111984,1.576577,0.002036,0.013115,1.923077
1154,Minnesota,Lyon,25857,13.977,33,7,0,5,0,0.027072,0.000000,0.000820,0.019337,0.000000
2314,Virginia,Caroline,29984,21.728,27,27,1,20,0,0.090048,3.703704,0.003335,0.066702,0.000000
167,California,Kern,905801,42.953,44,910,7,225,2,0.100464,0.769231,0.002283,0.024840,0.888889
2077,Tennessee,Warren,39839,35.539,31,7,0,4,0,0.017571,0.000000,0.000567,0.010040,0.000000
1715,Ohio,Mercer,40814,34.012,36,21,1,13,1,0.051453,4.761905,0.001429,0.031852,7.692308
1967,South Dakota,Clay,13864,12.993,32,6,0,5,0,0.043278,0.000000,0.001352,0.036065,0.000000
1406,Nebraska,Merrick,7802,6.212,27,12,0,9,0,0.153807,0.000000,0.005697,0.115355,0.000000
372,Georgia,Colquitt,46137,32.264,37,185,6,87,5,0.400980,3.243243,0.010837,0.188569,5.747126
2129,Texas,Donley,3311,1.374,28,24,0,24,0,0.724857,0.000000,0.025888,0.724857,0.000000


In [43]:
county_health = pd.read_csv('relevant_data/us-county-health-rankings-2020.csv')

In [44]:
county_health = county_health.dropna(subset=['county'])

In [45]:
county_health

,fips,state,county,num_deaths,years_of_potential_life_lost_rate,95percent_ci_low,95percent_ci_high,quartile,ypll_rate_aian,ypll_rate_aian_95percent_ci_low,...,percent_hispanic,num_non_hispanic_white,percent_non_hispanic_white,num_not_proficient_in_english,percent_not_proficient_in_english,95percent_ci_low_39,95percent_ci_high_39,percent_female,num_rural,percent_rural
1,1001,Alabama,Autauga,791.0,8128.591190,7283.340731,8973.841649,1.0,NaN,NaN,...,2.965774,41316,74.308016,426,0.820225,0.347891,1.292558,51.448715,22921.0,42.002162
2,1003,Alabama,Baldwin,2967.0,7354.122530,6918.554269,7789.690790,1.0,NaN,NaN,...,4.646779,181201,83.111337,1068,0.543517,0.347271,0.739763,51.538377,77060.0,42.279099
3,1005,Alabama,Barbour,472.0,10253.573403,8782.217281,11724.929524,2.0,NaN,NaN,...,4.276355,11356,45.641252,398,1.631683,0.824903,2.438462,47.216752,18613.0,67.789635
4,1007,Alabama,Bibb,471.0,11977.539484,10344.064842,13611.014126,3.0,NaN,NaN,...,2.625000,16708,74.589286,57,0.268210,0.000000,0.807504,46.781250,15663.0,68.352607
5,1009,Alabama,Blount,1085.0,11335.071134,10288.871387,12381.270881,3.0,NaN,NaN,...,9.571231,50255,86.886238,934,1.724520,1.198129,2.250911,50.726141,51562.0,89.951502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3188,56037,Wyoming,Sweetwater,532.0,7831.827655,6915.944384,8747.710925,4.0,NaN,NaN,...,16.083250,34145,79.312908,669,1.633221,0.993813,2.272629,48.544749,4782.0,10.916313
3189,56039,Wyoming,Teton,109.0,2730.603992,1991.644111,3469.563872,1.0,NaN,NaN,...,14.878038,18812,81.504268,945,4.314477,2.787340,5.841615,48.403449,9887.0,46.430920
3190,56041,Wyoming,Uinta,256.0,7330.683549,6011.148684,8650.218415,3.0,NaN,NaN,...,9.236908,17741,87.398394,133,0.696080,0.000000,1.429709,49.332479,9101.0,43.095937
3191,56043,Wyoming,Washakie,110.0,6586.049959,4500.924119,8671.175799,2.0,NaN,NaN,...,14.051997,6498,82.409639,25,0.324886,0.000000,1.376766,49.422955,3068.0,35.954529


In [46]:
county_health.columns[:100]

Index(['fips', 'state', 'county', 'num_deaths',
       'years_of_potential_life_lost_rate', '95percent_ci_low',
       '95percent_ci_high', 'quartile', 'ypll_rate_aian',
       'ypll_rate_aian_95percent_ci_low', 'ypll_rate_aian_95percent_ci_high',
       'ypll_rate_asian', 'ypll_rate_asian_95percent_ci_low',
       'ypll_rate_asian_95percent_ci_high', 'ypll_rate_black',
       'ypll_rate_black_95percent_ci_low', 'ypll_rate_black_95percent_ci_high',
       'ypll_rate_hispanic', 'ypll_rate_hispanic_95percent_ci_low',
       'ypll_rate_hispanic_95percent_ci_high', 'ypll_rate_white',
       'ypll_rate_white_95percent_ci_low', 'ypll_rate_white_95percent_ci_high',
       'percent_fair_or_poor_health', '95percent_ci_low_2',
       '95percent_ci_high_2', 'quartile_2',
       'average_number_of_physically_unhealthy_days', '95percent_ci_low_3',
       '95percent_ci_high_3', 'quartile_3',
       'average_number_of_mentally_unhealthy_days', '95percent_ci_low_4',
       '95percent_ci_high_4', 'quar

In [53]:
excluded_column_words = [
    'aian',
    'asian',
    'black',
    'hispanic',
    'white',
    'quartile',
    'ci_high',
    'ci_low',
    'unreliabe',
    'alaska',
    'hawaiian',
    'petitioned',
    'fips',
    'num',
    'unreliable',
    'denominator',
    'years_of_potential_life_lost_rate',
    'chlamydia_rate',
    'ratio',
    'population'
]


filtered_columns = county_health.columns[~county_health.columns.str.contains('|'.join(excluded_column_words))]

In [54]:
filtered_columns

Index(['state', 'county', 'percent_fair_or_poor_health',
       'percent_low_birthweight', 'percent_smokers',
       'percent_adults_with_obesity', 'food_environment_index',
       'percent_physically_inactive',
       'percent_with_access_to_exercise_opportunities',
       'percent_excessive_drinking',
       'percent_driving_deaths_with_alcohol_involvement', 'teen_birth_rate',
       'percent_uninsured', 'primary_care_physicians_rate', 'dentist_rate',
       'mental_health_provider_rate', 'preventable_hospitalization_rate',
       'percent_with_annual_mammogram', 'percent_vaccinated', 'cohort_size',
       'high_school_graduation_rate', 'percent_some_college', 'labor_force',
       'percent_unemployed', 'percent_children_in_poverty',
       '80th_percentile_income', '20th_percentile_income',
       'percent_single_parent_households', 'social_association_rate',
       'annual_average_violent_crimes', 'violent_crime_rate',
       'injury_death_rate', 'average_daily_pm2_5',
       'pres

In [55]:
len(filtered_columns)

76

In [56]:
filtered_county_health = county_health[filtered_columns]

In [57]:
filtered_county_health

,state,county,percent_fair_or_poor_health,percent_low_birthweight,percent_smokers,percent_adults_with_obesity,food_environment_index,percent_physically_inactive,percent_with_access_to_exercise_opportunities,percent_excessive_drinking,...,firearm_fatalities_rate,juvenile_arrest_rate,average_traffic_volume_per_meter_of_major_roadways,percent_homeowners,percent_severe_housing_cost_burden,percent_less_than_18_years_of_age,percent_65_and_over,percent_not_proficient_in_english,percent_female,percent_rural
1,Alabama,Autauga,20.882987,8.619529,18.081557,33.3,7.2,34.7,69.130124,15.026031,...,16.230078,10.625000,88.457040,74.894625,13.403481,23.674035,15.562670,0.820225,51.448715,42.002162
2,Alabama,Baldwin,17.509134,8.345003,17.489033,31.0,8.0,26.5,73.713549,17.958310,...,14.093514,26.192661,86.997430,73.619343,11.725808,21.607911,20.443350,0.543517,51.538377,42.279099
3,Alabama,Barbour,29.591802,11.474559,21.999985,41.7,5.6,23.5,53.166770,12.844016,...,17.761715,15.000000,102.291762,61.397779,14.059546,20.867328,19.420441,1.631683,47.216752,67.789635
4,Alabama,Bibb,19.439724,10.308710,19.114200,37.6,7.8,33.5,16.251364,15.570042,...,23.936170,NaN,29.335580,75.073099,9.886874,20.477679,16.473214,0.268210,46.781250,68.352607
5,Alabama,Blount,21.745293,7.604563,19.208672,33.8,8.4,30.3,15.634486,14.230594,...,20.072746,7.460317,33.411782,78.626214,8.462257,23.153527,18.236515,1.724520,50.726141,89.951502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3188,Wyoming,Sweetwater,14.813082,9.705248,18.073422,29.8,7.7,25.0,89.686344,22.871908,...,15.881227,15.961538,154.754714,75.370172,8.385595,26.224710,12.111217,1.633221,48.544749,10.916313
3189,Wyoming,Teton,11.914358,7.736721,14.546369,12.4,8.2,11.6,99.718230,25.533577,...,NaN,4.210526,135.188600,57.730946,10.769059,18.365755,15.415277,4.314477,48.403449,46.430920
3190,Wyoming,Uinta,15.537464,10.110206,17.212675,35.8,7.4,27.2,84.018373,19.113357,...,18.394276,8.148148,96.191604,75.268261,7.707668,28.789596,14.079511,0.696080,49.332479,43.095937
3191,Wyoming,Washakie,15.955971,7.028754,16.859400,28.8,8.3,28.2,83.182937,16.523261,...,NaN,15.000000,82.822155,76.826417,6.816780,22.739379,21.686747,0.324886,49.422955,35.954529


In [58]:
filtered_county_health.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3142 entries, 1 to 3192
Data columns (total 76 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   state                                               3142 non-null   object 
 1   county                                              3142 non-null   object 
 2   percent_fair_or_poor_health                         3142 non-null   float64
 3   percent_low_birthweight                             3035 non-null   float64
 4   percent_smokers                                     3142 non-null   float64
 5   percent_adults_with_obesity                         3142 non-null   float64
 6   food_environment_index                              3123 non-null   float64
 7   percent_physically_inactive                         3142 non-null   float64
 8   percent_with_access_to_exercise_opportunities       3136 non-null   float64
 9

In [59]:
complete_data = grouped_data.merge(
    filtered_county_health, left_on=['county', 'state'], right_on=['county', 'state']
)

In [60]:
complete_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2549 entries, 0 to 2548
Data columns (total 88 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   state                                               2549 non-null   object 
 1   county                                              2549 non-null   object 
 2   population                                          2549 non-null   int64  
 3   density_km                                          2549 non-null   float64
 4   days_counted                                        2549 non-null   int64  
 5   case_sum                                            2549 non-null   int64  
 6   death_sum                                           2549 non-null   int64  
 7   case_count_20_days                                  2549 non-null   int64  
 8   death_count_20_days                                 2549 non-null   int64  
 9

In [107]:
complete_data.dropna(thresh=2400, axis=1).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2549 entries, 0 to 2548
Data columns (total 73 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   state                                               2549 non-null   object 
 1   county                                              2549 non-null   object 
 2   population                                          2549 non-null   int64  
 3   density_km                                          2549 non-null   float64
 4   days_counted                                        2549 non-null   int64  
 5   case_sum                                            2549 non-null   int64  
 6   death_sum                                           2549 non-null   int64  
 7   case_count_20_days                                  2549 non-null   int64  
 8   death_count_20_days                                 2549 non-null   int64  
 9

In [108]:
final_data = complete_data.dropna(thresh=2400, axis=1)

In [109]:
final_data.shape

(2549, 73)

In [112]:
final_data.dropna()

,state,county,population,density_km,days_counted,case_sum,death_sum,case_count_20_days,death_count_20_days,confirmed_infection(%_of_pop),...,percent_enrolled_in_free_or_reduced_lunch,segregation_index_2,average_traffic_volume_per_meter_of_major_roadways,percent_homeowners,percent_severe_housing_cost_burden,percent_less_than_18_years_of_age,percent_65_and_over,percent_not_proficient_in_english,percent_female,percent_rural
0,Alabama,Autauga,54571,35.436,37,42,4,19,1,0.076964,...,43.408673,23.628395,88.457040,74.894625,13.403481,23.674035,15.562670,0.820225,51.448715,42.002162
1,Alabama,Baldwin,182265,44.261,47,174,3,25,1,0.095465,...,48.486565,31.825343,86.997430,73.619343,11.725808,21.607911,20.443350,0.543517,51.538377,42.279099
2,Alabama,Barbour,27457,11.979,27,39,1,29,0,0.142040,...,63.292302,23.449713,102.291762,61.397779,14.059546,20.867328,19.420441,1.631683,47.216752,67.789635
3,Alabama,Bibb,22915,14.215,31,42,0,26,0,0.183286,...,61.562116,52.600206,29.335580,75.073099,9.886874,20.477679,16.473214,0.268210,46.781250,68.352607
4,Alabama,Blount,57322,34.325,36,37,0,15,0,0.064548,...,52.906287,17.694495,33.411782,78.626214,8.462257,23.153527,18.236515,1.724520,50.726141,89.951502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2542,Wyoming,Park,28702,1.596,44,1,0,1,0,0.003484,...,34.536853,22.828463,85.954553,71.729148,8.323841,20.594053,23.206247,0.627835,50.194380,44.240383
2543,Wyoming,Sheridan,29596,4.529,50,12,0,8,0,0.040546,...,32.822946,19.720794,100.097335,67.900296,11.440945,21.373995,20.993616,0.348727,49.793272,35.478775
2545,Wyoming,Sweetwater,45267,1.676,37,11,0,7,0,0.024300,...,39.333579,25.352653,154.754714,75.370172,8.385595,26.224710,12.111217,1.633221,48.544749,10.916313
2547,Wyoming,Uinta,21025,3.899,28,6,0,6,0,0.028537,...,37.784558,11.577927,96.191604,75.268261,7.707668,28.789596,14.079511,0.696080,49.332479,43.095937


In [113]:
wrangled_data = final_data.dropna()

In [114]:
print(wrangled_data.columns)

Index(['state', 'county', 'population', 'density_km', 'days_counted',
       'case_sum', 'death_sum', 'case_count_20_days', 'death_count_20_days',
       'confirmed_infection(%_of_pop)', 'death_perc(%_of_case)',
       'avg_daily_case_increase(%_of_pop)', 'case_count_20_days(%_of_pop)',
       'death_count_20_days(%_of_case)', 'percent_fair_or_poor_health',
       'percent_low_birthweight', 'percent_smokers',
       'percent_adults_with_obesity', 'food_environment_index',
       'percent_physically_inactive',
       'percent_with_access_to_exercise_opportunities',
       'percent_excessive_drinking',
       'percent_driving_deaths_with_alcohol_involvement', 'teen_birth_rate',
       'percent_uninsured', 'primary_care_physicians_rate', 'dentist_rate',
       'mental_health_provider_rate', 'preventable_hospitalization_rate',
       'percent_with_annual_mammogram', 'percent_vaccinated',
       'high_school_graduation_rate', 'percent_some_college', 'labor_force',
       'percent_unemployed

In [115]:
wrangled_data = wrangled_data.drop(['population', 'case_sum', 'death_sum'], axis=1)

In [116]:
wrangled_data

,state,county,density_km,days_counted,case_count_20_days,death_count_20_days,confirmed_infection(%_of_pop),death_perc(%_of_case),avg_daily_case_increase(%_of_pop),case_count_20_days(%_of_pop),...,percent_enrolled_in_free_or_reduced_lunch,segregation_index_2,average_traffic_volume_per_meter_of_major_roadways,percent_homeowners,percent_severe_housing_cost_burden,percent_less_than_18_years_of_age,percent_65_and_over,percent_not_proficient_in_english,percent_female,percent_rural
0,Alabama,Autauga,35.436,37,19,1,0.076964,9.523810,0.002080,0.034817,...,43.408673,23.628395,88.457040,74.894625,13.403481,23.674035,15.562670,0.820225,51.448715,42.002162
1,Alabama,Baldwin,44.261,47,25,1,0.095465,1.724138,0.002031,0.013716,...,48.486565,31.825343,86.997430,73.619343,11.725808,21.607911,20.443350,0.543517,51.538377,42.279099
2,Alabama,Barbour,11.979,27,29,0,0.142040,2.564103,0.005261,0.105620,...,63.292302,23.449713,102.291762,61.397779,14.059546,20.867328,19.420441,1.631683,47.216752,67.789635
3,Alabama,Bibb,14.215,31,26,0,0.183286,0.000000,0.005912,0.113463,...,61.562116,52.600206,29.335580,75.073099,9.886874,20.477679,16.473214,0.268210,46.781250,68.352607
4,Alabama,Blount,34.325,36,15,0,0.064548,0.000000,0.001793,0.026168,...,52.906287,17.694495,33.411782,78.626214,8.462257,23.153527,18.236515,1.724520,50.726141,89.951502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2542,Wyoming,Park,1.596,44,1,0,0.003484,0.000000,0.000079,0.003484,...,34.536853,22.828463,85.954553,71.729148,8.323841,20.594053,23.206247,0.627835,50.194380,44.240383
2543,Wyoming,Sheridan,4.529,50,8,0,0.040546,0.000000,0.000811,0.027031,...,32.822946,19.720794,100.097335,67.900296,11.440945,21.373995,20.993616,0.348727,49.793272,35.478775
2545,Wyoming,Sweetwater,1.676,37,7,0,0.024300,0.000000,0.000657,0.015464,...,39.333579,25.352653,154.754714,75.370172,8.385595,26.224710,12.111217,1.633221,48.544749,10.916313
2547,Wyoming,Uinta,3.899,28,6,0,0.028537,0.000000,0.001019,0.028537,...,37.784558,11.577927,96.191604,75.268261,7.707668,28.789596,14.079511,0.696080,49.332479,43.095937


In [117]:
wrangled_data.to_csv('./relevant_data/wrangled_data.csv', index=False)

Maybe more data is better.